## Imports

In [75]:
# DO NOT CHANGE THESE LINES
using Suppressor
@suppress begin
    using DataFrames, CSV, Random, Statistics, Serialization, LazyJSON, StatsBase, DecisionTree, ScientificTypes, MLJ
end

## Paths

In [76]:
# DO NOT CHANGE THESE LINES
ROOT_DIR = dirname(pwd())
MODEL_INPUTS_OUTPUTS = joinpath(ROOT_DIR, "model_inputs_outputs")
INPUT_DIR = joinpath(MODEL_INPUTS_OUTPUTS, "inputs")
INPUT_SCHEMA_DIR = joinpath(INPUT_DIR, "schema")
DATA_DIR = joinpath(INPUT_DIR, "data")
TRAIN_DIR = joinpath(DATA_DIR, "training")
TEST_DIR = joinpath(DATA_DIR, "testing")
MODEL_PATH = joinpath(MODEL_INPUTS_OUTPUTS, "model")
MODEL_ARTIFACTS_PATH = joinpath(MODEL_PATH, "artifacts")
OHE_ENCODER_FILE = joinpath(MODEL_ARTIFACTS_PATH, "ohe.jld2")
PREDICTOR_DIR_PATH = joinpath(MODEL_ARTIFACTS_PATH, "predictor")
PREDICTOR_FILE_PATH = joinpath(PREDICTOR_DIR_PATH, "predictor.ser")
IMPUTATION_FILE = joinpath(MODEL_ARTIFACTS_PATH, "imputation.ser")
TOP_CATEGORIES = joinpath(MODEL_ARTIFACTS_PATH, "top_categories.ser")
TARGET_LEVELS = joinpath(MODEL_ARTIFACTS_PATH, "target_levels.ser")

if !isdir(MODEL_ARTIFACTS_PATH)
    mkdir(MODEL_ARTIFACTS_PATH)
end
if !isdir(PREDICTOR_DIR_PATH)
    mkdir(PREDICTOR_DIR_PATH)
end

### Reading the schema
The schema contains metadata about the datasets. We will use the scehma to get information about the type of each feature (NUMERIC or CATEGORICAL) and the id and target features, this will be helpful in preprocessing stage.

In [77]:
# Reading a schema from a JSON file and extracting features
file_name = first(filter(x -> endswith(x, "json"), readdir(INPUT_SCHEMA_DIR)))
schema_path = joinpath(INPUT_SCHEMA_DIR, file_name)
schema_string = read(schema_path, String)
schema = LazyJSON.parse(schema_string)

features = schema["features"]
numeric_features = String[]
categorical_features = String[]
nullable_features = String[]

for f in features
    if f["dataType"] == "CATEGORICAL"
        push!(categorical_features, f["name"])
    else
        push!(numeric_features, f["name"])
    end
    if f["nullable"]
        push!(nullable_features, f["name"])
    end
end

# Extracting ID and target features
id_feature = schema["id"]["name"]
target_feature = schema["target"]["name"]


"evaluation"

### Reading training data

In [78]:
# Read the CSV file into a DataFrame
file_name = first(filter(x -> endswith(x, ".csv"), readdir(TRAIN_DIR)))
file_path = joinpath(TRAIN_DIR, file_name)
train_df = DataFrame(CSV.File(file_path))

,id,buying,maint,doors,persons,lug_boot,safety,evaluation
,Int64,String7,String7,String7,String7,String7,String7,String7
1,1539,low,med,3,2,small,low,unacc
2,748,high,med,5more,more,small,med,unacc
3,1648,low,low,3,2,small,med,unacc
4,68,vhigh,vhigh,4,4,med,high,unacc
5,1273,med,low,5more,2,med,med,unacc
6,1540,low,med,3,2,small,med,unacc
7,712,high,med,4,4,small,med,unacc
8,836,high,low,4,more,big,high,acc
9,1128,med,med,3,more,med,low,unacc


## Data Preprocessing
Data preprocessing is very important before training the model, as the data may contain missing values in some cells. Moreover, most of the learning algorithms cannot work with categorical data, thus the data has to be encoded.

In this section we will impute the missing values and encode the categorical features. Afterwards the data will be ready to train the model.

##### Imputing missing data
> The median value will be used to impute missing values of the numeric features and the mode will be used to impute categorical features.

##### You can add your own preprocessing steps such as:
<ul>
<li>Normalization</li> <br>
<li>Outlier removal</li><br>
<li>Dropping or adding features</li><br>
</ul>

### Important note:
<p> 
Saving the values used for imputation during training step is crucial. These values will be used to impute missing data in the testing set. This is very important to avoid the well known problem of data leakage. During testing, you should not make any assumptions about the data in hand, alternatively anything needed during the testing phase should be learned from the training phase. This is why we are creating a dictionary of values used during training to reuse these values during testing.
</p>


In [79]:
# Imputing missing data
imputation_values = Dict{String, Any}()

for column in nullable_features
    if column in numeric_features
        value = median(skipmissing(train_df[!, column]))
    else
        value = mode(skipmissing(train_df[!, column]))
    end
    train_df[!, column] = coalesce.(train_df[!, column], value)
    imputation_values[column] = value
end

# Serialize the imputation_values dictionary to a binary file
open(IMPUTATION_FILE, "w") do io
    serialize(io, imputation_values)
end


##### Encoding Categorical features
<p>
The id column is just an identifier for the training example, so we will exclude it during the encoding phase.<br>
Target feature will be label encoded in the next step.
</p>


In [80]:
# Store the target column in a variable
target = train_df[:, target_feature]

# Drop the target column from the dataframe
select!(train_df, Not(target_feature))

# Function to get top 3 categories
function get_top_categories(df, features, n=3)
    top_cats = Dict()
    for feature in features
        col_data = df[!, feature]
        category_counts = StatsBase.countmap(col_data)
        sorted_categories = sort(collect(category_counts), by=x->x[2], rev=true)
        
        # Take minimum between n and the number of unique categories
        num_categories = min(n, length(sorted_categories))
        
        top_cats[feature] = [x[1] for x in sorted_categories[1:num_categories]]
    end
    return top_cats
end

# Get top 3 categories for specific features
top_categories = get_top_categories(train_df, categorical_features)


Dict{Any, Any} with 6 entries:
  "persons"  => String7["4", "2", "more"]
  "buying"   => String7["high", "vhigh", "low"]
  "safety"   => String7["med", "low", "high"]
  "doors"    => String7["2", "3", "5more"]
  "lug_boot" => String7["small", "big", "med"]
  "maint"    => String7["high", "med", "low"]

In [81]:
# Function to one-hot encode only the top 3 categories
function one_hot_top_categories!(df, top_categories)
    for (feature, top_cats) in top_categories
        if length(top_cats) == 2  # Handle the binary case
            # Assuming the first category in top_cats is treated as 'true'
            new_col_name = Symbol(string(feature, "_binary"))
            df[!, new_col_name] = df[!, feature] .== top_cats[1]
        else  # Handle the general case
            for cat in top_cats
                new_col_name = Symbol(string(feature, "_", cat))
                df[!, new_col_name] = df[!, feature] .== cat
            end
        end
        select!(df, Not(Symbol(feature)))  # Drop the original feature column
    end
end

# Apply one-hot encoding
one_hot_top_categories!(train_df, top_categories)

# Serialize the top_categories dictionary to a binary file
open(TOP_CATEGORIES, "w") do io
    serialize(io, top_categories)
end


### Training the Classifier
We choose Random Forest Classifier model, but feel free to try your own and compare the results.

In [82]:
x_train = Matrix(train_df[:, Not([id_feature])])

# train random forest classifier
# using 2 random features, 10 trees, 0.5 portion of samples per tree, and a maximum tree depth of 6
model = build_forest(string.(target), x_train, 2, 10, 0.5, 6)

open(PREDICTOR_FILE_PATH, "w") do io
    serialize(io, model)
end